# Updating an MKF file with background estimates in 5 bands

This notebook used the function ``mkfwork`` which calculates the background in 5 bands (using the KP-COR_SAX model of KCG) for each time in a specified mkf file, then writes out a new FITS file which includes a copy of the HDUs from the original MKF file, with a new HDU with the background rates included as columns.  The new HDU is called "PREFILTERB" (for now).

The bands used for the background rate calculation are:

* ``0000 <= PI <= 0025``
* ``0035 <= PI <= 0200``
* ``0200 <= PI <= 0800``
* ``0800 <= PI <= 1200``
* ``1200 <= PI <= 1500``


In [1]:
#from nicer import mkfwork 
from nicer.kcg import mkfwork
from nicer.nicer import nicerObs
from astropy.time import Time
from astropy.io.fits import table_to_hdu
from astropy.io import fits
from astropy.table import Table
rcParams['text.usetex']=False

%matplotlib inline

**In the cell below, define:**

``libdir`` = The directory where the background library is stored

``mkffile`` = The mkf file to be updated

In [2]:
libdir = '/Volumes/SXDC/Data/NICER/kp_model/gen10'
mkffile = '/Volumes/SXDC/Data/NICER/kp_model/h1743322kp.mkf'

In [ ]:
tstart= Time.now()
print (tstart.iso)

#mkf = mkfwork(mkffile, libdir=libdir)

tend = Time.now()
print(tend.iso)
print("Duration = {0:.2f} sec".format((tend.mjd-tstart.mjd)*86400))


In [ ]:
# Check the file

hdu = fits.open('h1743322kp_bkg.mkf')

In [ ]:
figure(figsize=[20,20])
subplot(531)
title('B_PI_0000_0025')
plot(hdu[-1].data['TIME'], hdu[-1].data['B_PI_0000_0025'])
subplot(532)
title('B_PI_0035_0200')
plot(hdu[-1].data['TIME'], hdu[-1].data['B_PI_0035_0200'])
subplot(533)
title('B_PI_0200_0800')
plot(hdu[-1].data['TIME'], hdu[-1].data['B_PI_0200_0800'])
subplot(534)
title('B_PI_0800_1200')
plot(hdu[-1].data['TIME'], hdu[-1].data['B_PI_0800_1200'])
subplot(535)
title('B_PI_1200_1500')
plot(hdu[-1].data['TIME'], hdu[-1].data['B_PI_1200_1500'])


# Background spectrum Creation from background event files

After discussing with Keith, it might be better to extract background spectra from the "combined" background event files.

One possible method:
* create a combined background event file for all the RXTE blank field observations
* create an MKF file that covers the times included in the combined event file (i.e. for the entire mission?)
 * this MKF file will need to have COR, SUN_ANGLE and KP values for every (1-second) time step (and any other parameter of importance)
* for a given observation:
 * get the good time intervals for the observation
 * for each gti:
   * get the KP, COR, SUN_ANGLE etc values during the gti
   * from the bkg event file, find:
     * distribution of bkg events in KP (from 0 to ...)
       * for each KP bin in the distribution (with  
     
extract all bkg events having those KP, COR... values and extract a spectrum from those events 
   
Keith has started on this:
* combined event file: ``/Volumes/SXDC/Data/NICER/kp_model/20181130/30nov18targskc.evt``
* combined mkf file: ``/Volumes/SXDC/Data/NICER/kp_model/20181130/30nov18targskckpclreduced.mkf``

In [ ]:
bevt = "/Volumes/SXDC/Data/NICER/kp_model/20181130/30nov18targskc.evt"
bmkf = "/Volumes/SXDC/Data/NICER/kp_model/20181130/30nov18targskckpclreduced.mkf"

In [ ]:
betab = Table.read(bevt, hdu=1)
bmtab = Table.read(bmkf, hdu=1)

In [3]:
bevt = "/Volumes/SXDC/Data/NICER/kp_model/20181130/30nov18targskc_enhanced.evt"
betab = Table.read(bevt, hdu=1)

In [4]:
bkgexpotot = fits.open(bevt)[1].header['EXPOSURE']
bkgexpotot

979721.215625912

## NOW Create a bkg spectrum for an observation




In [5]:
root = '/Volumes/SXDC/Data/NICER/etacar/repro'
rmf = '/Users/corcoran/Dropbox/nicer_cal/nicer_resp_ver1.02/nicer_v1.02.rmf'
arf = '/Users/corcoran/Dropbox/nicer_cal/nicer_resp_ver1.02/ni_xrcall_onaxis_v1.02.arf'
obsid = 1110010141
nobs = nicerObs(obsid,rootdir=root, rmffile=rmf, arffile=arf)
mkffile = '/Volumes/SXDC/Data/NICER/etacar/repro/1110010141/auxil/ni1110010141.mkf3'

In [6]:
nDF = nobs.get_eventsdf()

gti = nobs.get_gti()

gti.iloc[0]

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:1: TableReplaceWarning: converted column 'PHA_FAST' from integer to float
  """Entry point for launching an IPython kernel.
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:1: TableReplaceWarning: converted column 'PI_FAST' from integer to float
  """Entry point for launching an IPython kernel.


START                   1.36602e+08
STOP                    1.36602e+08
Duration                        727
STARTISO    2018-05-01 00:56:02.184
STOPISO     2018-05-01 01:08:09.184
Name: 0, dtype: object

In [7]:
# GET THE MKF3 INFO FOR THIS OBSERVATIONS
# mkf3 file from the observation

mkf3 = fits.open(mkffile)

# these are the times, kp, SA, cor and deadtime values for the GTI from the observation

mkfkp = mkf3['PREFILTER'].data.KP
mkftime = mkf3['PREFILTER'].data.TIME
mkfSA = mkf3['PREFILTER'].data.SUN_ANGLE
mkfcor = mkf3['PREFILTER'].data.COR_SAX
mkfDT = mkf3['PREFILTER'].data.MPU_DEADTIME.mean(axis=1)


In [42]:
for gtinum in arange(len(gti))[4:5]:
    print gtinum, t0s, t0e

4 136618369.0 136619142.0


In [43]:
# for a given GTI, find the start and stop times for the gti, then get the values of 
# KP, COR_SAX, SUN_Angle, deadtime within the GTI

#gtinum=4 # the selected gti number
for gtinum in arange(len(gti))[4:5]:

    t0s = gti.iloc[gtinum].START
    t0e = gti.iloc[gtinum].STOP

    # it are the indices which select the specified good time interval
    it = np.where((mkftime >= t0s) & (mkftime <= t0e) )[0]

    # now find time length of time that kp > kplo and kp < kphi
    gtidur=gti.loc[gtinum].Duration

    #select parameter values (kp, cor, sa, dt) corresponding to the selected GTI 
    mkfkpgti = mkf3['PREFILTER'].data.KP[it]
    mkftimegti = mkf3['PREFILTER'].data.TIME[it]
    mkfSAgti = mkf3['PREFILTER'].data.SUN_ANGLE[it]
    mkfcorgti = mkf3['PREFILTER'].data.COR_SAX[it]
    mkfDTgti = mkf3['PREFILTER'].data.MPU_DEADTIME.mean(axis=1)[it]


    # NOW SELECT BY KP values for the given gtinum

    a=np.histogram(mkfkpgti, bins=arange(0,15))
    kpnum = a[0]
    kpwt = kpnum/kpnum.sum() # kpwt is the fraction of the gti duration in that KP bin 
    kpbin = a[1]

    # for each non-zero kp bin get the fraction of the gti duration 
    # that the kp value was within that KP bin 

    #ik selects only those kpwt bins which are non-zero
    ik = np.where(kpwt > 0.0)[0]

    # for the selected KP bin - i = the selected KP bin
    i=0 
    ind = ik[i] # index of the i'th non-empty bin

    #  kpbin[i] gives the lower edge of the kp bin for the i'th (non-empty) bin
    #  kplo, kphi give the lower, upper bin edge
    #  for example ik[0] is the index in the kpbin of the first non-zero bin
    kplo = kpbin[ind]
    kphi = kpbin[ind+1]

    # since the mkf data are sampled once per second, the total time in this KP bin is just the 
    # length of the isel array * 1 second.  Then we have to multiply that time by the 
    # fraction of time the kp value was in that bin, i.e. by kpwt[i]
    isel = np.where((mkfkpgti >=kplo) & (mkfkpgti <=kphi))[0]

    #select parameter values (kp, cor, sa, dt) corresponding to the kp bin range
    mkfkpsel = mkf3['PREFILTER'].data.KP[isel]
    mkftimesel = mkf3['PREFILTER'].data.TIME[isel]
    mkfsasel = mkf3['PREFILTER'].data.SUN_ANGLE[isel]
    mkfcorsel = mkf3['PREFILTER'].data.COR_SAX[isel]
    mkfdtsel = mkf3['PREFILTER'].data.MPU_DEADTIME.mean(axis=1)[isel]

    # for now let's assume the times are continuous; 
    #  TODO: check for discontinuities 
    # to get the total exposure in this kp bin, get the total number of seconds = isel*0.0+1 (first term), 
    #   correct for Deadtime (second term), 
    #   and multiply by the fraction of total time in the selected kp bin (third term)
    expo = np.sum((isel*0.0+1.0)*(1.0-mkfDT[it[isel]])*kpwt[ik[i]])
    # if the exposure is more than gtidur, truncate it to gtidur
    # note: this should only be needed when the exposure is just a bit larger than gtidur; if it's much larger then there 
    # are problems!
    if expo > gtidur:
        expo = gtidur
    print("expo = {ex}; dur of GTI = {gtid}".format(ex=expo, gtid=gtidur))

    # so we've extracted the times when the KPs are within the first bin range; 
    # we NEXT need to extract BKG events from the bkg events that have KPs in the specified range,
    # then accumulate the events and the exposure time - 
    # (assume a single "GTI" of duration equal to the exposure time)

    betabselk = betab[(betab['KP']> kplo) & (betab['KP'] <=kphi)] # number of bkg events
    bhist = np.histogram(betab['KP'], bins = kpbin)[0]
    bexpowt = bhist[ik[i]]/float(bhist.sum()) # fraction of bkg exposure in this kp bin
    bdtave = betabselk['MPU_DEADTIME'].mean() # use the average deadtime for bkg exposure calculation
    bexpototk = bkgexpotot*(1-bdtave)*bexpowt

    # then we want to look at the distribution of cor
    # for all the cor values in the mkf3 for all values with kp within the selected kp bin,
    #    find the distribution of cor values
    #      then for each non-empty cor bin
    #        * calculate fraction of expo of time where cor is in that bin
    #        * extract bkg events from the kp-selected events which have cor values within that bin


    print(bexpototk, bkgexpotot)
    print("% of bkg events between {kpl} < kp < {kph} = {bper:.2f}%".format(kpl=kplo, kph=kphi, bper=bexpototk/bkgexpotot*100))

expo = 188.0; dur of GTI = 188.0
(359377.1206633965, 979721.215625912)
% of bkg events between 1 < kp < 2 = 36.68%


In [44]:
    #
    # NOW FOR COR; bexpotot is the total background exposure in the kp bin - now we need the fraction of that exposure
    # which has COR in the selected COR bin
    #

    # set the initial exposure to the KP bin exposure
    cexpo = expo

    corbins=arange(1, 20)
    a=np.histogram(mkfcorsel, bins = corbins)
    cornum = a[0]
    corwt = cornum/float(cornum.sum()) # corwt is the fraction of the gti duration in that cor bin 
    corbin = a[1]

    # for each non-zero kp bin get the fraction of the gti duration 
    # that the kp value was within that KP bin 

    #ik selects only those kpwt bins which are non-zero
    #ik = np.where(corwt > 0.0)[0]
    for ind in ik:
        #ind = ik[i] # index of the i'th non-empty bin
        print(ind)

        # bin ranges for this bin
        corlo = corbin[ind]
        corhi = corbin[ind+1]

        # since the mkf data are sampled once per second, the total time is just the 
        # length of the isel array * 1 second.  Then we have to multiply that time by the 
        # fraction of time the kp value was in that bin, i.e. by kpwt[i]
        isel = np.where((mkfcorsel >=corlo) & (mkfcorsel <=corhi))[0]

        # for now let's assume the times are continuous; TODO:check for discontinuities 
        # include Deadtime correction (second term) and fraction of total time in kp,cor bin (third term)
        cexpo = ((isel*0.0+1)*(1.0-mkfdtsel[isel])*corwt[ind]).sum()
        if cexpo > gtidur:
            cexpo = gtidur
        print("expo = {ex}; dur of GTI = {gtid}".format(ex=cexpo, gtid=gtidur))

        # so we've extracted the times when the KPs are within the first bin range; 
        # and the cor's are in their selected bin
        # we need to extract BKG events from the bkg events that have KPs and CORs in the specified bins,
        # then accumulate the bkg events and the bkg exposure time - 
        # (assume a single bkg "GTI" of duration equal to the exposure time)

        # number of bkg events in the selected kp, cor bin
        betabselc = betabselk[(betabselk['COR_SAX']> corlo) & (betabselk['COR_SAX'] <=corhi)] 
        bhist = np.histogram(betabselk['COR_SAX'], bins = corbin)[0]
        bexpowt = bhist[ind]/float(bhist.sum()) # fraction of bkg exposure in this kp, cor bin
        bdtave = betabselc['MPU_DEADTIME'].mean() # use the average deadtime for bkg exposure calculation
        bexpototc = bexpototk*(1.0-bdtave)*bexpowt

        # then we want to look at the distribution of cor
        # for all the cor values in the mkf3 for all values with kp within the selected kp bin,
        #    find the distribution of cor values
        #      then for each non-empty cor bin
        #        * calculate fraction of expo of time where cor is in that bin
        #        * extract bkg events from the kp-selected events which have cor values within that bin


        print(bexpototc, bexpototk, bkgexpotot)
        print("% of bkg events between {bl} < COR < {bh} = {bper:.2f}%".format(bl=corlo, bh=corhi, bper=bexpototc/bkgexpotot*100))

        print("Done COR index {0}; {1} < COR < {2}".format(i, corlo, corhi))

12
expo = 25.051505926; dur of GTI = 188.0
(15948.656357172536, 359377.1206633965, 979721.215625912)
% of bkg events between 13 < COR < 14 = 1.63%
Done COR index 0; 13 < COR < 14


In [45]:
ik

array([12, 13])

In [26]:
        #
        # NOW FOR SUN ANGLE
        #

        sexpo = cexpo

        sabin=arange(40,190, 10)
        a=np.histogram(mkfsasel, bins = sabin)
        #q = hist(mkfsasel, bins = sabin)
        sanum = a[0]
        sawt = sanum/float(sanum.sum()) #  the fraction of the gti duration in that SA bin 
        sabin = a[1]

        # for each non-zero kp bin get the fraction of the gti duration 
        # that the kp value was within that KP bin 

        #ik selects only those kpwt bins which are non-zero
        ik = np.where(sawt > 0.0)[0]
        betabselsa_arr = []
        bexpotot_arr=[]
        for ind in ik[:]:

            # FIRST GET THE SOURCE OBSERVATION EXPOSURE in this kp, cor, SA bin

            # bin ranges for this bin
            salo = sabin[ind]
            sahi = sabin[ind+1]

            # since the mkf data are sampled once per second, the total time is just the 
            # length of the isel array * 1 second.  Then we have to multiply that time by the 
            # fraction of time the kp value was in that bin, i.e. by kpwt[i]
            isel = np.where((mkfsasel >=salo) & (mkfsasel <=sahi))[0]

            # for now let's assume the times are continuous; TODO:check for discontinuities 
            # include Deadtime correction (second term) and fraction of total time in kp,cor bin (third term)
            sexpo = ((isel*0.0+1.0)*(1.0-mkfdtsel[isel])*sawt[ind]).sum()
            print(sexpo)
            if sexpo > gtidur:
                sexpo = gtidur
            print("expo = {ex}; dur of GTI = {gtid}".format(ex=sexpo, gtid=gtidur))

            # SECOND, EXTRACT THE BKG EVENTS and  EXPOSURE in this kp, cor, SA bin

            # so we've extracted the times when the KPs are within the first bin range; 
            # and the cor's are in their first bin
            # and are marching through the non-zero sun angle bins
            # we need to extract BKG events from the bkg events that have KPs and CORs and SUN_ANGLES in the specified range,
            # then accumulate the events and the exposure time - 
            # (assume a single "GTI" of duration equal to the exposure time)
            # betabsel is the background event table in the appropriate KP and COR range
            betabselsa = betabselc[(betabselc['SUN_ANGLE']> salo) & (betabselc['SUN_ANGLE'] <=sahi)] # number of bkg events in the specified SA bin
            print("for {0}<Sun Angle<{1}; Number of Events={2}".format(salo, sahi, len(betabselsa)))
            if len(betabselsa) > 0:
                betabselsa_arr.append(betabselsa)
                bhist = np.histogram(betabselc['SUN_ANGLE'], bins = sabin)[0]
                bexpowt = bhist[ind]/float(bhist.sum()) # fraction of bkg exposure in this kp, cor, sa bin
                bdtave = betabselsa['MPU_DEADTIME'].mean() # use the average deadtime for bkg exposure calculation
                bexpototsa = bexpototc*(1.0-bdtave)*bexpowt
                bexpotot_arr.append(bexpototsa)  
            else:
                print("No Events Found")
            print("Done SUN Angle index {0}; {1} < SUN_ANGLE < {2}; # Events = {3}".format(ind, 
                                                                                          salo, 
                                                                                          sahi,
                                                                                         len(betabselsa)))
            print("Source exposure = {0}; Background exposure = {1}\n".format(sexpo, bexpototsa))
        btotexpo_kcsa = np.asarray(bexpotot_arr).sum()
        print("total exposure in this kp, Cor bin for observed SA values = {0}".format(btotexpo_kcsa))




0.131295013346358
expo = 0.131295013346; dur of GTI = 188.0
for 50<Sun Angle<60; Number of Events=837
Done SUN Angle index 1; 50 < SUN_ANGLE < 60; # Events = 837
Source exposure = 0.131295013346; Background exposure = 511.962641906

0.5291005291005291
expo = 0.529100529101; dur of GTI = 188.0
for 60<Sun Angle<70; Number of Events=2799
Done SUN Angle index 2; 60 < SUN_ANGLE < 70; # Events = 2799
Source exposure = 0.529100529101; Background exposure = 1717.68895791

0.5291005291005291
expo = 0.529100529101; dur of GTI = 188.0
for 70<Sun Angle<80; Number of Events=3869
Done SUN Angle index 3; 70 < SUN_ANGLE < 80; # Events = 3869
Source exposure = 0.529100529101; Background exposure = 2430.33317925

0.5291005291005291
expo = 0.529100529101; dur of GTI = 188.0
for 80<Sun Angle<90; Number of Events=5359
Done SUN Angle index 4; 80 < SUN_ANGLE < 90; # Events = 5359
Source exposure = 0.529100529101; Background exposure = 3370.1735

0.7619047619047619
expo = 0.761904761905; dur of GTI = 188.0
fo

## So let's make a function to accumulate the background spectra

In [55]:
def mk_bkg_spec(root = '/Volumes/SXDC/Data/NICER/etacar/repro',
                rmf = '/Users/corcoran/Dropbox/nicer_cal/nicer_resp_ver1.02/nicer_v1.02.rmf',
                arf = '/Users/corcoran/Dropbox/nicer_cal/nicer_resp_ver1.02/ni_xrcall_onaxis_v1.02.arf',
                obsid = 1110010141,
                nobs = nicerObs(obsid,rootdir=root, rmffile=rmf, arffile=arf),
                mkffile = '/Volumes/SXDC/Data/NICER/etacar/repro/1110010141/auxil/ni1110010141.mkf3',
                gtimin = 100):
    # get gti's from the observation
    nDF = nobs.get_eventsdf()
    gti = nobs.get_gti()
    # GET THE MKF3 INFO FOR THIS OBSERVATIONS
    # mkf3 file from the observation
    mkf3 = fits.open(mkffile)
    # these are the times, kp, SA, cor and deadtime values for the GTI from the observation
    mkfkp = mkf3['PREFILTER'].data.KP
    mkftime = mkf3['PREFILTER'].data.TIME
    mkfSA = mkf3['PREFILTER'].data.SUN_ANGLE
    mkfcor = mkf3['PREFILTER'].data.COR_SAX
    mkfDT = mkf3['PREFILTER'].data.MPU_DEADTIME.mean(axis=1)

    # for a given GTI, find the start and stop times for the gti, then get the values of 
    # KP, COR_SAX, SUN_Angle, deadtime within the GTI

    for gtinum in arange(len(gti)):
        gdur = gti.iloc[gtinum].Duration
        print("\nFor GTI #{gn}; Duration = {d}\n".format(gn=gtinum,d = gdur))
        if gdur > gtimin:

            t0s = gti.iloc[gtinum].START
            t0e = gti.iloc[gtinum].STOP

            # it are the indices which select the specified good time interval
            it = np.where((mkftime >= t0s) & (mkftime <= t0e) )[0]

            # now find time length of time that kp > kplo and kp < kphi
            gtidur=gti.loc[gtinum].Duration

            #select parameter values (kp, cor, sa, dt) corresponding to the selected GTI 
            mkfkpgti = mkf3['PREFILTER'].data.KP[it]
            mkftimegti = mkf3['PREFILTER'].data.TIME[it]
            mkfSAgti = mkf3['PREFILTER'].data.SUN_ANGLE[it]
            mkfcorgti = mkf3['PREFILTER'].data.COR_SAX[it]
            mkfDTgti = mkf3['PREFILTER'].data.MPU_DEADTIME.mean(axis=1)[it]


            # NOW SELECT BY KP values for the given gtinum

            a=np.histogram(mkfkpgti, bins=arange(0,15))
            kpnum = a[0]
            kpwt = kpnum/kpnum.sum() # kpwt is the fraction of the gti duration in that KP bin 
            kpbin = a[1]

            # for each non-zero kp bin get the fraction of the gti duration 
            # that the kp value was within that KP bin 

            #ik selects only those kpwt bins which are non-zero
            ik = np.where(kpwt > 0.0)[0]

            # for the selected KP bin - i = the selected KP bin
            i=0 
            ind = ik[i] # index of the i'th non-empty bin

            #  kpbin[i] gives the lower edge of the kp bin for the i'th (non-empty) bin
            #  kplo, kphi give the lower, upper bin edge
            #  for example ik[0] is the index in the kpbin of the first non-zero bin
            kplo = kpbin[ind]
            kphi = kpbin[ind+1]

            # since the mkf data are sampled once per second, the total time in this KP bin is just the 
            # length of the isel array * 1 second.  Then we have to multiply that time by the 
            # fraction of time the kp value was in that bin, i.e. by kpwt[i]
            isel = np.where((mkfkpgti >=kplo) & (mkfkpgti <=kphi))[0]

            #select parameter values (kp, cor, sa, dt) corresponding to the kp bin range
            mkfkpsel = mkf3['PREFILTER'].data.KP[isel]
            mkftimesel = mkf3['PREFILTER'].data.TIME[isel]
            mkfsasel = mkf3['PREFILTER'].data.SUN_ANGLE[isel]
            mkfcorsel = mkf3['PREFILTER'].data.COR_SAX[isel]
            mkfdtsel = mkf3['PREFILTER'].data.MPU_DEADTIME.mean(axis=1)[isel]

            # for now let's assume the times are continuous; 
            #  TODO: check for discontinuities 
            # to get the total exposure in this kp bin, get the total number of seconds = isel*0.0+1 (first term), 
            #   correct for Deadtime (second term), 
            #   and multiply by the fraction of total time in the selected kp bin (third term)
            expo = np.sum((isel*0.0+1.0)*(1.0-mkfDT[it[isel]])*kpwt[ik[i]])
            # if the exposure is more than gtidur, truncate it to gtidur
            # note: this should only be needed when the exposure is just a bit larger than gtidur; if it's much larger then there 
            # are problems!
            if expo > gtidur:
                expo = gtidur
            print("expo = {ex}; dur of GTI = {gtid}".format(ex=expo, gtid=gtidur))

            # so we've extracted the times when the KPs are within the first bin range; 
            # we NEXT need to extract BKG events from the bkg events that have KPs in the specified range,
            # then accumulate the events and the exposure time - 
            # (assume a single "GTI" of duration equal to the exposure time)

            betabselk = betab[(betab['KP']> kplo) & (betab['KP'] <=kphi)] # number of bkg events
            bhist = np.histogram(betab['KP'], bins = kpbin)[0]
            bexpowt = bhist[ik[i]]/float(bhist.sum()) # fraction of bkg exposure in this kp bin
            bdtave = betabselk['MPU_DEADTIME'].mean() # use the average deadtime for bkg exposure calculation
            bexpototk = bkgexpotot*(1-bdtave)*bexpowt

            # then we want to look at the distribution of cor
            # for all the cor values in the mkf3 for all values with kp within the selected kp bin,
            #    find the distribution of cor values
            #      then for each non-empty cor bin
            #        * calculate fraction of expo of time where cor is in that bin
            #        * extract bkg events from the kp-selected events which have cor values within that bin


            print(bexpototk, bkgexpotot)
            print("% of bkg events between {kpl} < kp < {kph} = {bper:.2f}%".format(kpl=kplo, kph=kphi, bper=bexpototk/bkgexpotot*100))


            #
            # NOW FOR COR; bexpotot is the total background exposure in the kp bin - now we need the fraction of that exposure
            # which has COR in the selected COR bin
            #

            # set the initial exposure to the KP bin exposure
            cexpo = expo

            corbins=arange(1, 20)
            a=np.histogram(mkfcorsel, bins = corbins)
            cornum = a[0]
            corwt = cornum/float(cornum.sum()) # corwt is the fraction of the gti duration in that cor bin 
            corbin = a[1]

            # for each non-zero kp bin get the fraction of the gti duration 
            # that the kp value was within that KP bin 

            #ik selects only those kpwt bins which are non-zero
            #ik = np.where(corwt > 0.0)[0]
            for ind in ik:
                #ind = ik[i] # index of the i'th non-empty bin
                print(ind)

                # bin ranges for this bin
                corlo = corbin[ind]
                corhi = corbin[ind+1]

                # since the mkf data are sampled once per second, the total time is just the 
                # length of the isel array * 1 second.  Then we have to multiply that time by the 
                # fraction of time the kp value was in that bin, i.e. by kpwt[i]
                isel = np.where((mkfcorsel >=corlo) & (mkfcorsel <=corhi))[0]

                # for now let's assume the times are continuous; TODO:check for discontinuities 
                # include Deadtime correction (second term) and fraction of total time in kp,cor bin (third term)
                cexpo = ((isel*0.0+1)*(1.0-mkfdtsel[isel])*corwt[ind]).sum()
                if cexpo > gtidur:
                    cexpo = gtidur
                print("expo = {ex}; dur of GTI = {gtid}".format(ex=cexpo, gtid=gtidur))

                # so we've extracted the times when the KPs are within the first bin range; 
                # and the cor's are in their selected bin
                # we need to extract BKG events from the bkg events that have KPs and CORs in the specified bins,
                # then accumulate the bkg events and the bkg exposure time - 
                # (assume a single bkg "GTI" of duration equal to the exposure time)

                # number of bkg events in the selected kp, cor bin
                betabselc = betabselk[(betabselk['COR_SAX']> corlo) & (betabselk['COR_SAX'] <=corhi)] 
                bhist = np.histogram(betabselk['COR_SAX'], bins = corbin)[0]
                bexpowt = bhist[ind]/float(bhist.sum()) # fraction of bkg exposure in this kp, cor bin
                bdtave = betabselc['MPU_DEADTIME'].mean() # use the average deadtime for bkg exposure calculation
                bexpototc = bexpototk*(1.0-bdtave)*bexpowt

                # then we want to look at the distribution of cor
                # for all the cor values in the mkf3 for all values with kp within the selected kp bin,
                #    find the distribution of cor values
                #      then for each non-empty cor bin
                #        * calculate fraction of expo of time where cor is in that bin
                #        * extract bkg events from the kp-selected events which have cor values within that bin


                print(bexpototc, bexpototk, bkgexpotot)
                print("% of bkg events between {bl} < COR < {bh} = {bper:.2f}%".format(bl=corlo, bh=corhi, bper=bexpototc/bkgexpotot*100))

                print("Done COR index {0}; {1} < COR < {2}".format(i, corlo, corhi))


                #
                # NOW FOR SUN ANGLE
                #

                sexpo = cexpo

                sabin=arange(40,190, 10)
                a=np.histogram(mkfsasel, bins = sabin)
                #q = hist(mkfsasel, bins = sabin)
                sanum = a[0]
                sawt = sanum/float(sanum.sum()) #  the fraction of the gti duration in that SA bin 
                sabin = a[1]

                # for each non-zero kp bin get the fraction of the gti duration 
                # that the kp value was within that KP bin 

                #ik selects only those kpwt bins which are non-zero
                ik = np.where(sawt > 0.0)[0]
                betabselsa_arr = []
                bexpotot_arr=[]
                for ind in ik:

                    # FIRST GET THE SOURCE OBSERVATION EXPOSURE in this kp, cor, SA bin

                    # bin ranges for this bin
                    salo = sabin[ind]
                    sahi = sabin[ind+1]

                    # since the mkf data are sampled once per second, the total time is just the 
                    # length of the isel array * 1 second.  Then we have to multiply that time by the 
                    # fraction of time the kp value was in that bin, i.e. by kpwt[i]
                    isel = np.where((mkfsasel >=salo) & (mkfsasel <=sahi))[0]

                    # for now let's assume the times are continuous; TODO:check for discontinuities 
                    # include Deadtime correction (second term) and fraction of total time in kp,cor bin (third term)
                    sexpo = ((isel*0.0+1.0)*(1.0-mkfdtsel[isel])*sawt[ind]).sum()
                    print(sexpo)
                    if sexpo > gtidur:
                        sexpo = gtidur
                    print("expo = {ex}; dur of GTI = {gtid}".format(ex=sexpo, gtid=gtidur))

                    # SECOND, EXTRACT THE BKG EVENTS and  EXPOSURE in this kp, cor, SA bin

                    # so we've extracted the times when the KPs are within the first bin range; 
                    # and the cor's are in their first bin
                    # and are marching through the non-zero sun angle bins
                    # we need to extract BKG events from the bkg events that have KPs and CORs and SUN_ANGLES in the specified range,
                    # then accumulate the events and the exposure time - 
                    # (assume a single "GTI" of duration equal to the exposure time)
                    # betabsel is the background event table in the appropriate KP and COR range
                    betabselsa = betabselc[(betabselc['SUN_ANGLE']> salo) & (betabselc['SUN_ANGLE'] <=sahi)] # number of bkg events in the specified SA bin
                    print("for {0}<Sun Angle<{1}; Number of Events={2}".format(salo, sahi, len(betabselsa)))
                    if len(betabselsa) > 0:
                        betabselsa_arr.append(betabselsa)
                        bhist = np.histogram(betabselc['SUN_ANGLE'], bins = sabin)[0]
                        bexpowt = bhist[ind]/float(bhist.sum()) # fraction of bkg exposure in this kp, cor, sa bin
                        bdtave = betabselsa['MPU_DEADTIME'].mean() # use the average deadtime for bkg exposure calculation
                        bexpototsa = bexpototc*(1.0-bdtave)*bexpowt
                        bexpotot_arr.append(bexpototsa)  
                    else:
                        print("No Events Found")
                    print("Done SUN Angle index {0}; {1} < SUN_ANGLE < {2}; # Events = {3}".format(ind, 
                                                                                                  salo, 
                                                                                                  sahi,
                                                                                                 len(betabselsa)))
                    print("Source exposure = {0}; Background exposure = {1}\n".format(sexpo, bexpototsa))
                btotexpo_kcsa = np.asarray(bexpotot_arr).sum()
                print("total exposure in this kp, Cor bin for observed SA values = {0}".format(btotexpo_kcsa))
        else: # gti duration less than minimum
            print("GTI #{gn} < {gmin} seconds, skipping".format(gn=gtinum, gmin=gtimin))
        return btotexpo_kcsa


In [57]:
btexpo = mk_bkg_spec()

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:9: TableReplaceWarning: converted column 'PHA_FAST' from integer to float
  if __name__ == '__main__':
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:9: TableReplaceWarning: converted column 'PI_FAST' from integer to float
  if __name__ == '__main__':



For GTI #0; Duration = 727.0

expo = 725.401178286; dur of GTI = 727.0
(122424.72595304607, 979721.215625912)
% of bkg events between 0 < kp < 1 = 12.50%
0
expo = 0.0; dur of GTI = 727.0
(16647.638338558987, 122424.72595304607, 979721.215625912)
% of bkg events between 1 < COR < 2 = 1.70%
Done COR index 0; 1 < COR < 2
0.034086205387996796
expo = 0.034086205388; dur of GTI = 727.0
for 50<Sun Angle<60; Number of Events=2465
Done SUN Angle index 1; 50 < SUN_ANGLE < 60; # Events = 2465
Source exposure = 0.034086205388; Background exposure = 511.100307476

0.13736263736263737
expo = 0.137362637363; dur of GTI = 727.0
for 60<Sun Angle<70; Number of Events=18460
Done SUN Angle index 2; 60 < SUN_ANGLE < 70; # Events = 18460
Source exposure = 0.137362637363; Background exposure = 3825.70615417

0.13736263736263737
expo = 0.137362637363; dur of GTI = 727.0
for 70<Sun Angle<80; Number of Events=5144
Done SUN Angle index 3; 70 < SUN_ANGLE < 80; # Events = 5144
Source exposure = 0.137362637363; Ba

(359377.1206633965, 979721.215625912)
% of bkg events between 1 < kp < 2 = 36.68%
1
expo = 0.0; dur of GTI = 188.0
(44338.41775606911, 359377.1206633965, 979721.215625912)
% of bkg events between 2 < COR < 3 = 4.53%
Done COR index 0; 2 < COR < 3
0.131295013346358
expo = 0.131295013346; dur of GTI = 188.0
for 50<Sun Angle<60; Number of Events=493
Done SUN Angle index 1; 50 < SUN_ANGLE < 60; # Events = 493
Source exposure = 0.131295013346; Background exposure = 305.855305624

0.5291005291005291
expo = 0.529100529101; dur of GTI = 188.0
for 60<Sun Angle<70; Number of Events=18448
Done SUN Angle index 2; 60 < SUN_ANGLE < 70; # Events = 18448
Source exposure = 0.529100529101; Background exposure = 11379.023597

0.5291005291005291
expo = 0.529100529101; dur of GTI = 188.0
for 70<Sun Angle<80; Number of Events=15826
Done SUN Angle index 3; 70 < SUN_ANGLE < 80; # Events = 15826
Source exposure = 0.529100529101; Background exposure = 9970.66112357

0.5291005291005291
expo = 0.529100529101; dur 

for 120<Sun Angle<130; Number of Events=11960
Done SUN Angle index 8; 120 < SUN_ANGLE < 130; # Events = 11960
Source exposure = 0.511508951407; Background exposure = 2547.95085644

total exposure in this kp, Cor bin for observed SA values = 13415.8490116

For GTI #8; Duration = 763.0

expo = 761.105592937; dur of GTI = 763.0
(122424.72595304607, 979721.215625912)
% of bkg events between 0 < kp < 1 = 12.50%
0
expo = 0.0; dur of GTI = 763.0
(16647.638338558987, 122424.72595304607, 979721.215625912)
% of bkg events between 1 < COR < 2 = 1.70%
Done COR index 0; 1 < COR < 2
0.03248004911316972
expo = 0.0324800491132; dur of GTI = 763.0
for 50<Sun Angle<60; Number of Events=2465
Done SUN Angle index 1; 50 < SUN_ANGLE < 60; # Events = 2465
Source exposure = 0.0324800491132; Background exposure = 511.100307476

0.13089005235602094
expo = 0.130890052356; dur of GTI = 763.0
for 60<Sun Angle<70; Number of Events=18460
Done SUN Angle index 2; 60 < SUN_ANGLE < 70; # Events = 18460
Source exposure =

In [58]:
btexpo

## Binning the bkg spectrum

So for a given source good time interval (with tstart, tstop):
* Get the distribution of KP, COR_SAX and SUN_ANGLE within tstart, tstop

For example, let's say we've got a gti with tstop-tstart = 20 sec and  3 cor_sax bins (5, 6 7) with 10sec, 3sec, 7sec, respectively, of time during that bin as determined from the source mkf file.

Assumptions:
* the variation of COR_SAX is continuous during a GTI.  So if the GTI is 100 sec long and there's some interval in the background GTI where the COR_SAX range is appropriate for a source event, the duration of the appropriate cor_sax bkg interval is tcorstop - tcorstart, where the tcor's are the start and stop of the in-bound cor_sax interval in the bkg GTI
* the variation of SUN_ANGLE shows jitter around an average value
* KP changes every 3 hours which is long compared to most GTIs, so the interpolated KP changes continously within a GTI (but need to worry about discontinuities) 



In [ ]:
gtinum=1
t0s = gti.iloc[gtinum].START
t0e = gti.iloc[gtinum].STOP
it = np.where((mkftime >= t0s) & (mkftime <= t0e) )[0]
off =  int(gti.iloc[0].START)

fig = figure(figsize=[10,8])
subplot(3,3,1)
plot(mkftime[it]-off, mkfcor[it],'o',label='cor')
xlabel("Time-{o}".format(o=off))
legend()
subplot(3,3,2)
plot(mkftime[it]-off, mkfSA[it],'-o', label='sa')
plot([mkftime[it].min()-off,mkftime[it].max()-off],[mkfSA[it].mean(),mkfSA[it].mean()])
xlabel("Time-{o}".format(o=off))
legend()
subplot(3,3,3)
plot(mkftime[it]-off, mkfkp[it],'-o', label='kp')

fig.tight_layout()
legend()

print(mkfSA[it].mean())

In [ ]:
# get the Sun Angle distribution range in bins of 1 degree for specified gti (here gtinum=1, i.e. the second gti)
# use this distribtution to determine 
gtinum=1
t0s = gti.iloc[gtinum].START
t0e = gti.iloc[gtinum].STOP
it = np.where((mkftime >= t0s) & (mkftime <= t0e) )[0]

b  = hist(mkfSA[it],label='sa', bins=arange(60,181))

In [ ]:
gtinum=6

ts = gti.iloc[gtinum].START
te = gti.iloc[gtinum].STOP

ylabel('KP')
plot(mkftime, mkfkp,'.')
plot([ts, ts],[0,1])
plot([te, te],[0,1])

In [ ]:
gti.Duration[11]